In [ ]:
# LOAD the data
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

!time pip install rdkit-pypi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

real	0m3.071s
user	0m2.732s
sys	0m0.305s


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sn
# import scipy as sp

import random
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv2D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

from rdkit import Chem # A core definition
from rdkit.Chem.Draw import MolsToGridImage # For displaying multiple molecules
from rdkit.Chem.AllChem import * # conformer generation and adding H 
from rdkit.Chem.rdMolDescriptors import * # To calculate descriptors
from rdkit.Chem.Draw import IPythonConsole # This displays the molecule in-line
from rdkit.Chem import PandasTools # for pandas dataframe with rdkit
from rdkit.DataStructs.cDataStructs import ConvertToNumpyArray
import pandas.io.formats.style

In [ ]:
############################################Sexual data
#Optional: move to the desired location:
%cd /content/drive/MyDrive/PhD/PTS_datasets/
!ls '/content/drive/MyDrive/PhD/PTS_datasets/'
import pandas as pd
sxl= pd.read_csv("M_sexual_LibrariesC_hits_nd_2ndclst_nhits.csv")
sxl['S_Hit'] = sxl['S_Hit'].replace(np.nan, "No")
sxl = sxl.iloc[: , 1:]
print(sxl['S_Hit'].value_counts())
sxl

/content/drive/MyDrive/PhD/PTS_datasets
 ABSActiveInactiveFeatCorrelationOnABSActives.csv
 ABSActiveInactiveFeatCorrelationOnABSInactives.csv
 absdualGBMFinalizedhpt5005_UMFP_model_prediction_on_pbox.csv
 absdualLRFinalizedhpt5005_UMFP_model_prediction_on_pbox.csv
 absdualRFFinalizedhpt5005_UMFP_model_prediction_on_pbox.csv
 absdualsvmFinalizedhpt5005_UMFP_model_prediction_on_pbox.csv
 asexualsFeaturesZtestprop.csv
 asexualsFeaturesZtestpropInact.csv
 asexualsFeaturesZtestproportionsActvsInactv.csv
 asexualsFeaturesZtestpropTopABSfeat.csv
 asexualsFeaturesZtestpropTopDualfeat.csv
 asexualsFeaturesZtestpropTopfeat.csv
 asexualsGBMhpt_UMFP_model5atom500bitFinal.pkl
 asexualsLogisticRegressionFinalHPT_UMFP_model5005.pkl
 asexualsSVCFinalHPT_UMFP_model5005.pkl
 axldeepinsighOverUnderXttestMFP.npy
 axldeepinsighOverUnderXttrainMFP.npy
 axldeepinsighOverUnderYttestMFP.npy
 axldeepinsighOverUnderYttrainMFP.npy
 axl_UnderMFP_GBM_HPT5005_FeatureImp.csv
 axl_UnderMFP_LR_HPT_FeatureImp.csv
 axl_U

,Smiles,S_Hit
0,NC(=O)c1cccc2nc([C@H]3CCN(Cc4ccccn4)C3)[nH]c12,Yes
1,Fc1ccc2[nH]c(C3CCCN3Cc3c[nH]cn3)nc2c1,Yes
2,COc1c(C)cnc(CN(C)C2CCCCC2)c1C,Yes
3,Cc1cccc(CN2CCCC2c2cccc(Nc3nccs3)n2)c1,Yes
4,CCSCCNc1nc(-c2ccccn2)nc2c1c(C)nn2C,Yes
...,...,...
9886,Cc1nc(C)n(C2CCCN(C(=O)c3ncoc3C(C)C)C2)n1,No
9887,COCCN1CCN(Cc2c(C)noc2C)Cc2ccc(C)nc21,No
9888,CCc1ncnc2c1CCN(Cc1ccc3c(c1)OCO3)CC2,No
9889,CCC1CN(CC(=O)NCCN2CCCCCC2)c2ccccc2S1,No


# MACCSKeysFingerprint

In [ ]:
!pip install deepchem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.2/693.2 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 65.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1


In [ ]:
import deepchem as dc
from deepchem.feat import MACCSKeysFingerprint

In [ ]:
sxl["Target"] = np.where(sxl["S_Hit"]== "Yes", 1,0)
sxl["Target"].value_counts()

0    8975
1     916
Name: Target, dtype: int64

In [ ]:
dchm= sxl["Smiles"]
dchm

0       NC(=O)c1cccc2nc([C@H]3CCN(Cc4ccccn4)C3)[nH]c12
1                Fc1ccc2[nH]c(C3CCCN3Cc3c[nH]cn3)nc2c1
2                        COc1c(C)cnc(CN(C)C2CCCCC2)c1C
3                Cc1cccc(CN2CCCC2c2cccc(Nc3nccs3)n2)c1
4                   CCSCCNc1nc(-c2ccccn2)nc2c1c(C)nn2C
                             ...                      
9886          Cc1nc(C)n(C2CCCN(C(=O)c3ncoc3C(C)C)C2)n1
9887              COCCN1CCN(Cc2c(C)noc2C)Cc2ccc(C)nc21
9888               CCc1ncnc2c1CCN(Cc1ccc3c(c1)OCO3)CC2
9889              CCC1CN(CC(=O)NCCN2CCCCCC2)c2ccccc2S1
9890                Cc1cnn(CC2CCCN2Cc2nc3ccccc3nc2C)c1
Name: Smiles, Length: 9891, dtype: object

In [ ]:
mols = [Chem.MolFromSmiles(x) for x in dchm]
featurizer = MACCSKeysFingerprint()
maccs = featurizer.featurize(mols)
maccs

array([[0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [ ]:
df=[]
df = pd.DataFrame(maccs)
#df["Target"]= sxl["Target"]
df

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,1,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,1,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,1,1,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9886,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
9887,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
9888,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
9889,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,1,1,0


In [ ]:
from sklearn.model_selection import train_test_split
Xt_train, Xt_test, Yt_train, Yt_test = train_test_split(df, sxl["Target"],test_size=0.2, random_state=42)

In [ ]:
######################################################################Penalized_SVM (https://www.analyticsvidhya.com/blog/2020/07/10-techniques-to-deal-with-class-imbalance-in-machine-learning/)

# load library
from sklearn.svm import SVC

# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC(class_weight='balanced', probability=True)

svc_model.fit(Xt_train, Yt_train)

svc_predict = svc_model.predict(Xt_test)# check performance


In [ ]:
# Save Model Using joblib
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

%cd /content/
# save the model to disk
filename = 'sexualsPenalizedSVM_UMACCS_model.sav'
joblib.dump(svc_model, filename)
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(Xt_test, Yt_test)
print(result)

/content
0.953511874684184


In [ ]:
!cp sexualsPenalizedSVM_UMACCS_model.sav /content/drive/MyDrive/PhD/PenalizedSVM_MACCS

In [ ]:
###Import model
#Optional: move to the desired location:
%cd /content/drive/MyDrive/PhD/PenalizedSVM_MACCS
!ls '/content/drive/MyDrive/PhD/PenalizedSVM_MACCS'

/content/drive/MyDrive/PhD/PenalizedSVM_MACCS
sexualsPenalizedSVM_UMACCS_model.sav


In [ ]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

# load the model from disk
loaded_model = joblib.load('/content/drive/MyDrive/PhD/PenalizedSVM_MACCS/sexualsPenalizedSVM_UMACCS_model.sav')
result = loaded_model.score(Xt_test, Yt_test)
print(result)

0.953511874684184


In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Yt_test,svc_predict)
print('Accuracy: %f' % accuracy)
recall= recall_score(Yt_test,svc_predict)
print("Recall %f" % recall)
precision= precision_score(Yt_test,svc_predict)
print("precision %f" % precision)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Yt_test,svc_predict)
print('F1 score: %f' % f1)
cm= confusion_matrix(Yt_test,svc_predict)
cm

Accuracy: 0.953512
Recall 0.869110
precision 0.712446
F1 score: 0.783019


array([[1721,   67],
       [  25,  166]])

In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score
print('ROCAUC score:',roc_auc_score(Yt_test, svc_predict))
print('Accuracy score:',accuracy_score(Yt_test, svc_predict))
print('F1 score:',f1_score(Yt_test, svc_predict))

ROCAUC score: 0.9158189559248978
Accuracy score: 0.953511874684184
F1 score: 0.7830188679245284


In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score, cohen_kappa_score
# kappa
kappa = cohen_kappa_score(Yt_test, svc_predict)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(Yt_test, svc_predict)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(Yt_test, svc_predict)
print(matrix)

Cohens kappa: 0.757272
ROC AUC: 0.915819
[[1721   67]
 [  25  166]]


In [ ]:
print(f"Test Set: {len(Xt_test)}")
print(Yt_test.value_counts())
print(matrix)

Test Set: 1979
0    1788
1     191
Name: Target, dtype: int64
[[1721   67]
 [  25  166]]


In [ ]:
##################################################Cross validation kfold=10

# load library
from sklearn.svm import SVC

# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC(class_weight='balanced', probability=True)

fit= svc_model.fit(Xt_train, Yt_train)

svc_predict = svc_model.predict(Xt_test)# check performance

from sklearn.model_selection import cross_val_score
scores = cross_val_score(svc_model,Xt_train, Yt_train, cv=10)

In [ ]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.96 accuracy with a standard deviation of 0.01


In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Yt_test,svc_predict)
print('Accuracy: %f' % accuracy)
recall= recall_score(Yt_test,svc_predict)
print("Recall %f" % recall)
precision= precision_score(Yt_test,svc_predict)
print("precision %f" % precision)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Yt_test,svc_predict)
print('F1 score: %f' % f1)
cm= confusion_matrix(Yt_test,svc_predict)
cm

Accuracy: 0.953512
Recall 0.869110
precision 0.712446
F1 score: 0.783019


array([[1721,   67],
       [  25,  166]])

In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score
print('ROCAUC score:',roc_auc_score(Yt_test, svc_predict))
print('Accuracy score:',accuracy_score(Yt_test, svc_predict))
print('F1 score:',f1_score(Yt_test, svc_predict))

ROCAUC score: 0.9158189559248978
Accuracy score: 0.953511874684184
F1 score: 0.7830188679245284


In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score, cohen_kappa_score
# kappa
kappa = cohen_kappa_score(Yt_test, svc_predict)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(Yt_test, svc_predict)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(Yt_test, svc_predict)
print(matrix)

Cohens kappa: 0.757272
ROC AUC: 0.915819
[[1721   67]
 [  25  166]]


In [ ]:
print(f"Test Set: {len(Xt_test)}")
print(Yt_test.value_counts())
print(matrix)

Test Set: 1979
0    1788
1     191
Name: Target, dtype: int64
[[1721   67]
 [  25  166]]


# Hyperparameter tuning


---



---



In [ ]:
## hyper-parameters (like what C or gamma values to use)
# load library
from sklearn.svm import SVC

# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC(class_weight='balanced', probability=True)

In [ ]:
kernels = ['Polynomial', 'RBF', 'Sigmoid','Linear']#A function which returns the corresponding SVC model
def getClassifier(ktype):
    if ktype == 0:
        # Polynomial kernal
        return SVC(kernel='poly', degree=8, gamma="auto",class_weight='balanced', probability=True)
    elif ktype == 1:
        # Radial Basis Function kernal
        return SVC(kernel='rbf', gamma="auto", class_weight='balanced', probability=True)
    elif ktype == 2:
        # Sigmoid kernal
        return SVC(kernel='sigmoid', gamma="auto", class_weight='balanced', probability=True)
    elif ktype == 3:
        # Linear kernal
        return SVC(kernel='linear', gamma="auto", class_weight='balanced', probability=True)

In [ ]:
## https://www.vebuso.com/2020/03/svm-hyperparameter-tuning-using-gridsearchcv/
import pandas as pd  
import numpy as np  
from sklearn.svm import SVC  
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

for i in range(4):
    # Separate data into test and training sets
    X_train, X_test, y_train, y_test = train_test_split(df, sxl["Target"],test_size=0.2, random_state=42)# Train a SVC model using different kernal
    svclassifier = getClassifier(i) 
    svclassifier.fit(X_train, y_train)# Make prediction
    y_pred = svclassifier.predict(X_test)# Evaluate our model
    print("Evaluation:", kernels[i], "kernel")
    print(classification_report(y_test,y_pred))

Evaluation: Polynomial kernel
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1788
           1       0.10      1.00      0.18       191

    accuracy                           0.10      1979
   macro avg       0.05      0.50      0.09      1979
weighted avg       0.01      0.10      0.02      1979



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation: RBF kernel
              precision    recall  f1-score   support

           0       0.99      0.93      0.96      1788
           1       0.56      0.88      0.68       191

    accuracy                           0.92      1979
   macro avg       0.77      0.90      0.82      1979
weighted avg       0.95      0.92      0.93      1979

Evaluation: Sigmoid kernel
              precision    recall  f1-score   support

           0       0.98      0.90      0.94      1788
           1       0.48      0.87      0.62       191

    accuracy                           0.90      1979
   macro avg       0.73      0.89      0.78      1979
weighted avg       0.94      0.90      0.91      1979

Evaluation: Linear kernel
              precision    recall  f1-score   support

           0       0.98      0.92      0.95      1788
           1       0.54      0.86      0.67       191

    accuracy                           0.92      1979
   macro avg       0.76      0.89      0.81      197

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
 
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}
 
grid = GridSearchCV(SVC(class_weight='balanced', probability=True), param_grid,cv=KFold(n_splits=10), refit = True, verbose = 3)
 
# fitting the model for grid search
grid.fit(Xt_train, Yt_train)

Fitting 10 folds for each of 25 candidates, totalling 250 fits
[CV 1/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.894 total time=  54.0s
[CV 2/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.915 total time=  44.9s
[CV 3/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.896 total time=  41.0s
[CV 4/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.914 total time=  42.9s
[CV 5/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.913 total time=  49.0s
[CV 6/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.932 total time=  48.4s
[CV 7/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.930 total time=  47.9s
[CV 8/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.917 total time=  43.7s
[CV 9/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.913 total time=  43.0s
[CV 10/10] END ......C=0.1, gamma=1, kernel=rbf;, score=0.919 total time=  41.9s
[CV 1/10] END .....C=0.1, gamma=0.1, kernel=rbf;, score=0.933 total time=  23.6s
[CV 2/10] END .....C=0.1, gamma=0.1, kernel=rb

In [ ]:

grid_predictions = grid.predict(Xt_test)
 
# print classification report
print(classification_report(Yt_test, grid_predictions))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      1788
           1       0.86      0.80      0.83       191

    accuracy                           0.97      1979
   macro avg       0.92      0.89      0.91      1979
weighted avg       0.97      0.97      0.97      1979



In [ ]:
print(grid.best_params_)

{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}


SVC(C=1000, class_weight='balanced', gamma=0.01, probability=True)

In [ ]:
######################################################################Penalized_SVM (https://www.analyticsvidhya.com/blog/2020/07/10-techniques-to-deal-with-class-imbalance-in-machine-learning/)

# load library
from sklearn.svm import SVC

#svm_weights = (clf.coef_ ** 2).sum(axis=0) # coef_ is only available when using a linear kernel

# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC(kernel='rbf', C= 1000, gamma= 0.01,
                class_weight='balanced', probability=True)

svc_model.fit(Xt_train, Yt_train)

svc_predict = svc_model.predict(Xt_test)# check performance


In [ ]:
##################################################Cross validation kfold=10

svc_predict = svc_model.predict(Xt_test)# check performance

from sklearn.model_selection import cross_val_score
scores = cross_val_score(svc_model,Xt_train, Yt_train, cv=10)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
print(scores.mean())
print(scores.std())
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score
print('ROCAUC score:',roc_auc_score(Yt_test, svc_predict))
print('Accuracy score:',accuracy_score(Yt_test, svc_predict))
print('F1 score:',f1_score(Yt_test, svc_predict))

from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score, cohen_kappa_score
# kappa
kappa = cohen_kappa_score(Yt_test, svc_predict)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(Yt_test, svc_predict)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(Yt_test, svc_predict)
print(matrix)
print(f"Test Set: {len(Xt_test)}")
print(Yt_test.value_counts())
print(matrix)

0.97 accuracy with a standard deviation of 0.01
0.9744682922780268
0.005178339137897622
ROCAUC score: 0.900166321140354
Accuracy score: 0.9717028802425467
F1 score: 0.8469945355191256
Cohens kappa: 0.831437
ROC AUC: 0.900166
[[1768   20]
 [  36  155]]
Test Set: 1979
0    1788
1     191
Name: Target, dtype: int64
[[1768   20]
 [  36  155]]


In [ ]:
# Save Model Using joblib
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

%cd /content/
# save the model to disk
filename = 'sexualsPenalizedSVM_HPTFinal_UMACCS_model.pkl'
joblib.dump(svc_model, filename)
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(Xt_test, Yt_test)
print(result)

/content
0.9717028802425467


In [ ]:
!cp 'sexualsPenalizedSVM_HPTFinal_UMACCS_model.pkl' /content/drive/MyDrive/PhD/PenalizedSVM_MACCS

In [ ]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

# load the model from disk
loaded_model = joblib.load('/content/drive/MyDrive/PhD/PenalizedSVM_MACCS/sexualsPenalizedSVM_HPTFinal_UMACCS_model.pkl')
result = loaded_model.score(Xt_test, Yt_test)
print(result)

0.9717028802425467


In [ ]:
# Save Model Using joblib
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

%cd /content/
# save the model to disk
filename = 'sexualsPenalizedSVM_HPTFinal_UMACCS_model.sav'
joblib.dump(svc_model, filename)
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(Xt_test, Yt_test)
print(result)

/content
0.9717028802425467


In [ ]:
!cp sexualsPenalizedSVM_HPTFinal_UMACCS_model.sav /content/drive/MyDrive/PhD/PenalizedSVM_MACCS

In [ ]:
###Import model
#Optional: move to the desired location:
%cd /content/drive/MyDrive/PhD/PenalizedSVM_MACCS
!ls '/content/drive/MyDrive/PhD/PenalizedSVM_MACCS'

/content/drive/MyDrive/PhD/PenalizedSVM_MACCS
sexualsPenalizedSVM_HPTFinal_UMACCS_model.sav
sexualsPenalizedSVM_UMACCS_model.sav


In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Yt_test,svc_predict)
print('Accuracy: %f' % accuracy)
recall= recall_score(Yt_test,svc_predict)
print("Recall %f" % recall)
precision= precision_score(Yt_test,svc_predict)
print("precision %f" % precision)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Yt_test,svc_predict)
print('F1 score: %f' % f1)
cm= confusion_matrix(Yt_test,svc_predict)
cm

In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Yt_test,svc_predict)
print('Accuracy: %f' % accuracy)
recall= recall_score(Yt_test,svc_predict)
print("Recall %f" % recall)
precision= precision_score(Yt_test,svc_predict)
print("precision %f" % precision)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Yt_test,svc_predict)
print('F1 score: %f' % f1)
cm= confusion_matrix(Yt_test,svc_predict)
cm

Accuracy: 0.971703
Recall 0.811518
precision 0.885714
F1 score: 0.846995


array([[1768,   20],
       [  36,  155]])

In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score
print('ROCAUC score:',roc_auc_score(Yt_test, svc_predict))
print('Accuracy score:',accuracy_score(Yt_test, svc_predict))
print('F1 score:',f1_score(Yt_test, svc_predict))

ROCAUC score: 0.900166321140354
Accuracy score: 0.9717028802425467
F1 score: 0.8469945355191256


In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score, cohen_kappa_score
# kappa
kappa = cohen_kappa_score(Yt_test, svc_predict)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(Yt_test, svc_predict)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(Yt_test, svc_predict)
print(matrix)

Cohens kappa: 0.831437
ROC AUC: 0.900166
[[1768   20]
 [  36  155]]


In [ ]:
print(f"Test Set: {len(Xt_test)}")
print(Yt_test.value_counts())
print(matrix)

Test Set: 1979
0    1788
1     191
Name: Target, dtype: int64
[[1768   20]
 [  36  155]]


In [ ]:
print(scores.mean())
print(scores.std())

0.9744682922780268
0.005178339137897622


# Hyperparameter tuning


---



---



In [ ]:
## hyper-parameters (like what C or gamma values to use)
# load library
from sklearn.svm import SVC

# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC( probability=True)

In [ ]:
kernels = ['Polynomial', 'RBF', 'Sigmoid','Linear']#A function which returns the corresponding SVC model
def getClassifier(ktype):
    if ktype == 0:
        # Polynomial kernal
        return SVC(kernel='poly', degree=8, gamma="auto", probability=True)
    elif ktype == 1:
        # Radial Basis Function kernal
        return SVC(kernel='rbf', gamma="auto", probability=True)
    elif ktype == 2:
        # Sigmoid kernal
        return SVC(kernel='sigmoid', gamma="auto", probability=True)
    elif ktype == 3:
        # Linear kernal
        return SVC(kernel='linear', gamma="auto", probability=True)

In [ ]:
## https://www.vebuso.com/2020/03/svm-hyperparameter-tuning-using-gridsearchcv/
import pandas as pd  
import numpy as np  
from sklearn.svm import SVC  
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

for i in range(4):
    # Separate data into test and training sets
    X_train, X_test, y_train, y_test = train_test_split(df, dpchm['hit'],test_size=0.2, random_state=42)# Train a SVC model using different kernal
    svclassifier = getClassifier(i) 
    svclassifier.fit(X_train, y_train)# Make prediction
    y_pred = svclassifier.predict(X_test)# Evaluate our model
    print("Evaluation:", kernels[i], "kernel")
    print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
 
# defining parameter range
param_grid = {'C': [0.1, 1, 10],
              'gamma': [1, 0.1, 0.01],
              'kernel': ['rbf']}

# defining parameter range
#param_grid = {'C': [0.1, 1, 10],
#              'gamma': [ 0.1, 0.01],
#              'kernel': ['rbf']}
 
grid = GridSearchCV(SVC( probability=True), param_grid, refit = True, verbose = 3)
 
# fitting the model for grid search
grid.fit(Xt_train, Yt_train)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
 
# defining parameter range
#param_grid = {'C': [0.1, 1, 10],
#              'gamma': [1, 0.1, 0.01],
#              'kernel': ['rbf']}

# defining parameter range
param_grid = {'C': [0.1, 1, 10],
              'gamma': [ 0.1, 0.01],
              'kernel': ['rbf']}
 
grid = GridSearchCV(SVC(class_weight='balanced', probability=True), param_grid, refit = True, verbose = 3)
 
# fitting the model for grid search
grid.fit(Xt_train, Yt_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.928 total time=26.4min
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.929 total time=26.2min
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.927 total time=26.9min
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.930 total time=27.9min
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.929 total time=28.3min
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.903 total time=20.8min
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.906 total time=17.4min
[CV 3/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.904 total time=14.7min
[CV 4/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.904 total time=15.1min
[CV 5/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.906 total time=15.2min


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
 
# defining parameter range
#param_grid = {'C': [0.1, 1, 10],
#              'gamma': [1, 0.1, 0.01],
#              'kernel': ['rbf']}

# defining parameter range
param_grid = {'C': [ 1, 10],
              'gamma': [ 0.1, 0.01],
              'kernel': ['rbf']}
 
grid = GridSearchCV(SVC(class_weight='balanced', probability=True), param_grid, cv=2, refit = True, verbose = 3)
 
# fitting the model for grid search
grid.fit(Xt_train, Yt_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV 1/2] END ........C=1, gamma=0.1, kernel=rbf;, score=0.957 total time=11.4min
[CV 2/2] END ........C=1, gamma=0.1, kernel=rbf;, score=0.956 total time=14.4min
[CV 1/2] END .......C=1, gamma=0.01, kernel=rbf;, score=0.919 total time= 6.3min
[CV 2/2] END .......C=1, gamma=0.01, kernel=rbf;, score=0.919 total time= 6.1min
[CV 1/2] END .......C=10, gamma=0.1, kernel=rbf;, score=0.961 total time=20.8min
[CV 2/2] END .......C=10, gamma=0.1, kernel=rbf;, score=0.958 total time=20.0min
[CV 1/2] END ......C=10, gamma=0.01, kernel=rbf;, score=0.941 total time= 3.7min
[CV 2/2] END ......C=10, gamma=0.01, kernel=rbf;, score=0.940 total time= 3.7min


In [ ]:

grid_predictions = grid.predict(Xt_test)
 
# print classification report
print(classification_report(Yt_test, grid_predictions))

In [ ]:
######################################################################Penalized_SVM (https://www.analyticsvidhya.com/blog/2020/07/10-techniques-to-deal-with-class-imbalance-in-machine-learning/)

# load library
from sklearn.svm import SVC

#svm_weights = (clf.coef_ ** 2).sum(axis=0) # coef_ is only available when using a linear kernel

# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC(kernel='rbf', C= 10, gamma= 0.1, probability=True)

svc_model.fit(Xt_train, Yt_train)

svc_predict = svc_model.predict(Xt_test)# check performance


In [ ]:
##################################################Cross validation kfold=10

svc_predict = svc_model.predict(Xt_test)# check performance

from sklearn.model_selection import cross_val_score
#scores = cross_val_score(svc_model,Xt_train, Yt_train, cv=10)
#print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
#print(scores.mean())
#print(scores.std())
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score
print('ROCAUC score:',roc_auc_score(Yt_test, svc_predict))
print('Accuracy score:',accuracy_score(Yt_test, svc_predict))
print('F1 score:',f1_score(Yt_test, svc_predict))

from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score, cohen_kappa_score
# kappa
kappa = cohen_kappa_score(Yt_test, svc_predict)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(Yt_test, svc_predict)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(Yt_test, svc_predict)
print(matrix)
print(f"Test Set: {len(Xt_test)}")
print(Yt_test.value_counts())
print(matrix)

ROCAUC score: 0.9762323560059292
Accuracy score: 0.9760624895010919
F1 score: 0.9760966199781934
Cohens kappa: 0.952132
ROC AUC: 0.976232
[[5802   68]
 [ 217 5819]]
Test Set: 11906
1    6036
0    5870
Name: hit, dtype: int64
[[5802   68]
 [ 217 5819]]


In [ ]:
# Save Model Using joblib
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

%cd /content/
# save the model to disk
filename = 'asexualsPenalizedSVM_HPTFinal_UMACCS_model.pkl'
joblib.dump(svc_model, filename)
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(Xt_test, Yt_test)
print(result)

/content
0.9760624895010919


In [ ]:
!cp 'asexualsPenalizedSVM_HPTFinal_UMACCS_model.pkl' /content/drive/MyDrive/PhD/PenalizedSVM_MACCS

In [ ]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

# load the model from disk
loaded_model = joblib.load('/content/drive/MyDrive/PhD/PenalizedSVM_MACCS/asexualsPenalizedSVM_HPTFinal_UMACCS_model.pkl')
result = loaded_model.score(Xt_test, Yt_test)
print(result)

In [ ]:
# load library
from sklearn.svm import SVC
# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC(kernel='rbf', C= 10, gamma= 0.1,
                class_weight='balanced', probability=True)

In [ ]:
##################################################Cross validation kfold=10
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

# load the model from disk
svc_model = joblib.load('/content/drive/MyDrive/PhD/PenalizedSVM_MACCS/asexualsPenalizedSVM_HPTFinal_UMACCS_model.sav')
svc_predict = svc_model.predict(Xt_test)# check performance

from sklearn.model_selection import cross_val_score
scores = cross_val_score(svc_model,Xt_train, Yt_train, cv=10)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.97 accuracy with a standard deviation of 0.00


In [ ]:
print(scores.mean())
print(scores.std())

0.9746135490895054
0.0017791645422492572


In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Yt_test,svc_predict)
print('Accuracy: %f' % accuracy)
recall= recall_score(Yt_test,svc_predict)
print("Recall %f" % recall)
precision= precision_score(Yt_test,svc_predict)
print("precision %f" % precision)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Yt_test,svc_predict)
print('F1 score: %f' % f1)
cm= confusion_matrix(Yt_test,svc_predict)
cm

Accuracy: 0.976230
Recall 0.964380
precision 0.988453
F1 score: 0.976268


array([[5802,   68],
       [ 215, 5821]])

In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score
print('ROCAUC score:',roc_auc_score(Yt_test, svc_predict))
print('Accuracy score:',accuracy_score(Yt_test, svc_predict))
print('F1 score:',f1_score(Yt_test, svc_predict))

ROCAUC score: 0.9763980286368105
Accuracy score: 0.9762304720309087
F1 score: 0.9762683438155138


In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score, cohen_kappa_score
# kappa
kappa = cohen_kappa_score(Yt_test, svc_predict)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(Yt_test, svc_predict)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(Yt_test, svc_predict)
print(matrix)

Cohens kappa: 0.952468
ROC AUC: 0.976398
[[5802   68]
 [ 215 5821]]


In [ ]:
print(f"Test Set: {len(Xt_test)}")
print(Yt_test.value_counts())
print(matrix)

Test Set: 11906
1    6036
0    5870
Name: hit, dtype: int64
[[5802   68]
 [ 215 5821]]


## Asexual stage

In [ ]:
# LOAD the data
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Optional: move to the desired location:
%cd /content/drive/MyDrive/PhD/PTS_datasets/
!ls '/content/drive/MyDrive/PhD/PTS_datasets/'
import pandas as pd
axl= pd.read_csv("M_asexuals_lib_hits_nd_clst_nhts.csv")
axl['Hit'] = axl['Hit'].replace(np.nan, "No")
axl = axl.iloc[: , 1:]
print(axl['Hit'].value_counts())
axl

In [ ]:
compounds =[]
df1= axl[:20000]

for _, smiles, hit in df1[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))

df2= axl[20001:23766]

for _, smiles, hit in df2[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))

df3= axl[23767:25396]

for _, smiles, hit in df3[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))


df4= axl[25397:25662]

for _, smiles, hit in df4[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))

df5= axl[25663:26912]

for _, smiles, hit in df5[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))

df6= axl[26913:28399]

for _, smiles, hit in df6[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))

df7= axl[28400:]

for _, smiles, hit in df7[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))

len(compounds)

In [ ]:
axl["Target"] = np.where(axl["Hit"]== "Yes", 1,0)
axl["Target"].value_counts()

In [ ]:
compounds
dpchm= pd.DataFrame (compounds, columns = ['mol', 'hit']) 
dpchm

In [ ]:
#dchm= axl["Smiles"]
#dchm
dpchm["Target"] = np.where(dpchm["hit"]== "Yes", 1,0)
dpchm["Target"].value_counts()

dchm= dpchm["mol"]
dchm

In [ ]:
!pip install deepchem


In [ ]:
import deepchem as dc
from deepchem.feat import MACCSKeysFingerprint
#mols = [Chem.MolFromSmiles(x) for x in dchm]
featurizer = MACCSKeysFingerprint()
maccs = featurizer.featurize(dchm)
maccs

In [ ]:
df=[]
df = pd.DataFrame(maccs)
#df["Target"]= sxl["Target"]
df

In [ ]:
dpchm['hit']= np.where(dpchm['hit']== "Yes", 1,0)
dpchm['hit']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

Xt_train, Xt_test, Yt_train, Yt_test = train_test_split(df, dpchm['hit'],test_size=0.2, random_state=42)

In [ ]:
######################################################################Penalized_SVM (https://www.analyticsvidhya.com/blog/2020/07/10-techniques-to-deal-with-class-imbalance-in-machine-learning/)

# load library
from sklearn.svm import SVC

# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC(class_weight='balanced', probability=True)

svc_model.fit(Xt_train, Yt_train)

svc_predict = svc_model.predict(Xt_test)# check performance


In [ ]:
##################################################Cross validation kfold=10

# load library
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
scores = cross_val_score(svc_model,Xt_train, Yt_train, cv=10)

In [ ]:
print("%0.4f accuracy with a standard deviation of %0.4f" % (scores.mean(), scores.std()))

0.9450 accuracy with a standard deviation of 0.0021


In [ ]:
# Save Model Using joblib
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

%cd /content/
# save the model to disk
filename = 'asexualsPenalizedSVM_UMACCS_model.sav'
joblib.dump(svc_model, filename)
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(Xt_test, Yt_test)
print(result)

/content
0.942633966067529


In [ ]:
!cp asexualsPenalizedSVM_UMACCS_model.sav /content/drive/MyDrive/PhD/PenalizedSVM_MACCS

In [ ]:
###Import model
#Optional: move to the desired location:
%cd /content/drive/MyDrive/PhD/PenalizedSVM_MACCS
!ls '/content/drive/MyDrive/PhD/PenalizedSVM_MACCS'

/content/drive/MyDrive/PhD/PenalizedSVM_MACCS
asexualsPenalizedSVM_UMACCS_model.sav
sexualsPenalizedSVM_HPTFinal_UMACCS_model.sav
sexualsPenalizedSVM_UMACCS_model.sav
sexualsPenalizedzLinearHPTSVM_UMACCS_model.sav
sxl_UnderMACCS_LinearSVM_HPT_FeatureImp.csv


In [ ]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

# load the model from disk
loaded_model = joblib.load('/content/drive/MyDrive/PhD/PenalizedSVM_MACCS/asexualsPenalizedSVM_UMACCS_model.sav')
result = loaded_model.score(Xt_test, Yt_test)
print(result)

0.942633966067529


In [ ]:
svc_predict = loaded_model.predict(Xt_test)

In [ ]:
svc_predict= np.where(svc_predict== "Yes", 1,0)
Yt_test= np.where(Yt_test== "Yes", 1,0)

In [ ]:
svc_predict
Yt_test

array([0, 1, 0, ..., 1, 0, 1])

In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Yt_test,svc_predict)
print('Accuracy: %f' % accuracy)
recall= recall_score(Yt_test,svc_predict)
print("Recall %f" % recall)
precision= precision_score(Yt_test,svc_predict)
print("precision %f" % precision)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Yt_test,svc_predict)
print('F1 score: %f' % f1)
cm= confusion_matrix(Yt_test,svc_predict)
cm

Accuracy: 0.942634
Recall 0.920974
precision 0.964267
F1 score: 0.942124


array([[5664,  206],
       [ 477, 5559]])

In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score
print('ROCAUC score:',roc_auc_score(Yt_test, svc_predict))
print('Accuracy score:',accuracy_score(Yt_test, svc_predict))
print('F1 score:',f1_score(Yt_test, svc_predict))

ROCAUC score: 0.9429402291531899
Accuracy score: 0.942633966067529
F1 score: 0.9421235488517922


In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score, cohen_kappa_score
# kappa
kappa = cohen_kappa_score(Yt_test, svc_predict)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(Yt_test, svc_predict)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(Yt_test, svc_predict)
print(matrix)

Cohens kappa: 0.885318
ROC AUC: 0.942940
[[5664  206]
 [ 477 5559]]


In [ ]:
print(f"Test Set: {len(Xt_test)}")
print(Yt_test.value_counts())
print(matrix)

In [ ]:
##################################################Cross validation kfold=10

# load library
from sklearn.svm import SVC

# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC(class_weight='balanced', probability=True)

fit= svc_model.fit(Xt_train, Yt_train)

svc_predict = svc_model.predict(Xt_test)# check performance

from sklearn.model_selection import cross_val_score
scores = cross_val_score(svc_model,Xt_train, Yt_train, cv=10)

In [ ]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.95 accuracy with a standard deviation of 0.00


# Hyperparameter tuning


---



---



In [ ]:
## hyper-parameters (like what C or gamma values to use)
# load library
from sklearn.svm import SVC

# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC( probability=True)

In [ ]:
kernels = ['Polynomial', 'RBF', 'Sigmoid','Linear']#A function which returns the corresponding SVC model
def getClassifier(ktype):
    if ktype == 0:
        # Polynomial kernal
        return SVC(kernel='poly', degree=8, gamma="auto", probability=True)
    elif ktype == 1:
        # Radial Basis Function kernal
        return SVC(kernel='rbf', gamma="auto", probability=True)
    elif ktype == 2:
        # Sigmoid kernal
        return SVC(kernel='sigmoid', gamma="auto", probability=True)
    elif ktype == 3:
        # Linear kernal
        return SVC(kernel='linear', gamma="auto", probability=True)

In [ ]:
## https://www.vebuso.com/2020/03/svm-hyperparameter-tuning-using-gridsearchcv/
import pandas as pd  
import numpy as np  
from sklearn.svm import SVC  
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

for i in range(4):
    # Separate data into test and training sets
    X_train, X_test, y_train, y_test = train_test_split(df, dpchm['hit'],test_size=0.2, random_state=42)# Train a SVC model using different kernal
    svclassifier = getClassifier(i) 
    svclassifier.fit(X_train, y_train)# Make prediction
    y_pred = svclassifier.predict(X_test)# Evaluate our model
    print("Evaluation:", kernels[i], "kernel")
    print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
 
# defining parameter range
param_grid = {'C': [0.1, 1, 10],
              'gamma': [1, 0.1, 0.01],
              'kernel': ['rbf']}

# defining parameter range
#param_grid = {'C': [0.1, 1, 10],
#              'gamma': [ 0.1, 0.01],
#              'kernel': ['rbf']}
 
grid = GridSearchCV(SVC( probability=True), param_grid, refit = True, verbose = 3)
 
# fitting the model for grid search
grid.fit(Xt_train, Yt_train)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
 
# defining parameter range
#param_grid = {'C': [0.1, 1, 10],
#              'gamma': [1, 0.1, 0.01],
#              'kernel': ['rbf']}

# defining parameter range
param_grid = {'C': [0.1, 1, 10],
              'gamma': [ 0.1, 0.01],
              'kernel': ['rbf']}
 
grid = GridSearchCV(SVC(class_weight='balanced', probability=True), param_grid, refit = True, verbose = 3)
 
# fitting the model for grid search
grid.fit(Xt_train, Yt_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.928 total time=26.4min
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.929 total time=26.2min
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.927 total time=26.9min
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.930 total time=27.9min
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.929 total time=28.3min
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.903 total time=20.8min
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.906 total time=17.4min
[CV 3/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.904 total time=14.7min
[CV 4/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.904 total time=15.1min
[CV 5/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.906 total time=15.2min


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
 
# defining parameter range
#param_grid = {'C': [0.1, 1, 10],
#              'gamma': [1, 0.1, 0.01],
#              'kernel': ['rbf']}

# defining parameter range
param_grid = {'C': [ 1, 10],
              'gamma': [ 0.1, 0.01],
              'kernel': ['rbf']}
 
grid = GridSearchCV(SVC(class_weight='balanced', probability=True), param_grid, cv=2, refit = True, verbose = 3)
 
# fitting the model for grid search
grid.fit(Xt_train, Yt_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV 1/2] END ........C=1, gamma=0.1, kernel=rbf;, score=0.957 total time=11.4min
[CV 2/2] END ........C=1, gamma=0.1, kernel=rbf;, score=0.956 total time=14.4min
[CV 1/2] END .......C=1, gamma=0.01, kernel=rbf;, score=0.919 total time= 6.3min
[CV 2/2] END .......C=1, gamma=0.01, kernel=rbf;, score=0.919 total time= 6.1min
[CV 1/2] END .......C=10, gamma=0.1, kernel=rbf;, score=0.961 total time=20.8min
[CV 2/2] END .......C=10, gamma=0.1, kernel=rbf;, score=0.958 total time=20.0min
[CV 1/2] END ......C=10, gamma=0.01, kernel=rbf;, score=0.941 total time= 3.7min
[CV 2/2] END ......C=10, gamma=0.01, kernel=rbf;, score=0.940 total time= 3.7min


In [ ]:

grid_predictions = grid.predict(Xt_test)
 
# print classification report
print(classification_report(Yt_test, grid_predictions))

In [ ]:
######################################################################Penalized_SVM (https://www.analyticsvidhya.com/blog/2020/07/10-techniques-to-deal-with-class-imbalance-in-machine-learning/)

# load library
from sklearn.svm import SVC

#svm_weights = (clf.coef_ ** 2).sum(axis=0) # coef_ is only available when using a linear kernel

# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC(kernel='rbf', C= 10, gamma= 0.1, probability=True)

svc_model.fit(Xt_train, Yt_train)

svc_predict = svc_model.predict(Xt_test)# check performance


In [ ]:
##################################################Cross validation kfold=10

svc_predict = svc_model.predict(Xt_test)# check performance

from sklearn.model_selection import cross_val_score
#scores = cross_val_score(svc_model,Xt_train, Yt_train, cv=10)
#print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
#print(scores.mean())
#print(scores.std())
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score
print('ROCAUC score:',roc_auc_score(Yt_test, svc_predict))
print('Accuracy score:',accuracy_score(Yt_test, svc_predict))
print('F1 score:',f1_score(Yt_test, svc_predict))

from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score, cohen_kappa_score
# kappa
kappa = cohen_kappa_score(Yt_test, svc_predict)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(Yt_test, svc_predict)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(Yt_test, svc_predict)
print(matrix)
print(f"Test Set: {len(Xt_test)}")
print(Yt_test.value_counts())
print(matrix)

ROCAUC score: 0.9762323560059292
Accuracy score: 0.9760624895010919
F1 score: 0.9760966199781934
Cohens kappa: 0.952132
ROC AUC: 0.976232
[[5802   68]
 [ 217 5819]]
Test Set: 11906
1    6036
0    5870
Name: hit, dtype: int64
[[5802   68]
 [ 217 5819]]


In [ ]:
# Save Model Using joblib
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

%cd /content/
# save the model to disk
filename = 'asexualsPenalizedSVM_HPTFinal_UMACCS_model.pkl'
joblib.dump(svc_model, filename)
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(Xt_test, Yt_test)
print(result)

/content
0.9760624895010919


In [ ]:
!cp 'asexualsPenalizedSVM_HPTFinal_UMACCS_model.pkl' /content/drive/MyDrive/PhD/PenalizedSVM_MACCS

In [ ]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

# load the model from disk
loaded_model = joblib.load('/content/drive/MyDrive/PhD/PenalizedSVM_MACCS/asexualsPenalizedSVM_HPTFinal_UMACCS_model.pkl')
result = loaded_model.score(Xt_test, Yt_test)
print(result)

In [ ]:
# load library
from sklearn.svm import SVC
# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC(kernel='rbf', C= 10, gamma= 0.1,
                class_weight='balanced', probability=True)

In [ ]:
##################################################Cross validation kfold=10
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

# load the model from disk
svc_model = joblib.load('/content/drive/MyDrive/PhD/PenalizedSVM_MACCS/asexualsPenalizedSVM_HPTFinal_UMACCS_model.pkl')
svc_predict = svc_model.predict(Xt_test)# check performance

from sklearn.model_selection import cross_val_score
scores = cross_val_score(svc_model,Xt_train, Yt_train, cv=10)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.97 accuracy with a standard deviation of 0.00


In [ ]:
print(scores.mean())
print(scores.std())

0.9746135490895054
0.0017791645422492572


In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Yt_test,svc_predict)
print('Accuracy: %f' % accuracy)
recall= recall_score(Yt_test,svc_predict)
print("Recall %f" % recall)
precision= precision_score(Yt_test,svc_predict)
print("precision %f" % precision)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Yt_test,svc_predict)
print('F1 score: %f' % f1)
cm= confusion_matrix(Yt_test,svc_predict)
cm

Accuracy: 0.976230
Recall 0.964380
precision 0.988453
F1 score: 0.976268


array([[5802,   68],
       [ 215, 5821]])

In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score
print('ROCAUC score:',roc_auc_score(Yt_test, svc_predict))
print('Accuracy score:',accuracy_score(Yt_test, svc_predict))
print('F1 score:',f1_score(Yt_test, svc_predict))

ROCAUC score: 0.9763980286368105
Accuracy score: 0.9762304720309087
F1 score: 0.9762683438155138


In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score, cohen_kappa_score
# kappa
kappa = cohen_kappa_score(Yt_test, svc_predict)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(Yt_test, svc_predict)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(Yt_test, svc_predict)
print(matrix)

Cohens kappa: 0.952468
ROC AUC: 0.976398
[[5802   68]
 [ 215 5821]]


In [ ]:
print(f"Test Set: {len(Xt_test)}")
print(Yt_test.value_counts())
print(matrix)

Test Set: 11906
1    6036
0    5870
Name: hit, dtype: int64
[[5802   68]
 [ 215 5821]]
